In [1]:
# enable camera ## capture live image ## generate embeeding ## compare the image from dataset ## final outcome

In [2]:
import cv2
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity            # Import all libraries and modules
from deepface import DeepFace

In [3]:
# function to enable camera and capture image
def capture_image():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")  # for face detection
    video_cap = cv2.VideoCapture(0)                                                                  # enable the camera  # run time enable
    while(1):                                                                                        # camera on permanently, condition always true ,infinite loop
        ret, video = video_cap.read()                                                                # read the image
        color = cv2.cvtColor(video, cv2.COLOR_BGR2GRAY)                                              # convert the image in BGR channel
        faces = face_cascade.detectMultiScale(color, scaleFactor = 1.1,minNeighbors = 5, minSize = (50, 50),flags = cv2.CASCADE_SCALE_IMAGE)  # (color, reduced image size at each scale (1.1 - 10%), minmimum size of the face( 30 refers to minimum width * 30 refers to minimum height),Fine tuning)  
    
        for(x,y,w,h) in faces:                                                                       # ( w - width of rectangle,  h - height of rectangle)
            cv2.rectangle(video, (x,y), (x+w, y+h),(0, 255,0),2)                                     # (image, cordinate of top left corner, coordinates of bottom right corner, BGR format color, thickness of rectangle)
        cv2.imshow("video_live", video)                                                              # to create a frame
        if cv2.waitKey(5) == ord("s"):                                                               # stop by pressing 's'
            image_path = os.path.join(r"C:\Users\hp\Desktop\live data", "live1.jpg")                 # provide path to the image
            cv2.imwrite(image_path, video)
            video_cap.release()
            return image_path

In [4]:
# function to check blurrness

def check_blurr(image_path, threshold = 100):           # creat a function to check the image
    image = cv2.imread(image_path)
    lap = cv2.Laplacian(image, cv2.CV_64F)              # compute the laplacian of an image
    var = lap.var()                                     # store the variance into var variable

    return var < threshold, var                         # return variance

In [5]:
from datetime import datetime
import pandas as pd

In [6]:
def mark_attendance(image_names, similarity):
        attendance_record = {
        'Date': datetime.now().strftime('%Y-%m-%d'),
        'Image Name': image_names,
        'Similarity Score': similarity,
        'Attendance Status': 'Present' #if similarity >= threshold else 'Absent'
    }
        Record = r"C:\Users\hp\Desktop\Attendance_Record.xlsx"
        current = pd.read_excel(r"C:\Users\hp\Desktop\Attendance_Record.xlsx")               # function to mark the attendance
        attendance_df = pd.DataFrame([attendance_record])
        updated_df = pd.concat([current, attendance_df])
    
        updated_df.to_excel(r"C:\Users\hp\Desktop\Attendance_Record.xlsx")
        print("Attendance marked successfully!")

In [8]:
image_path = capture_image()                                                                         # capture the image
blurry, variance = check_blurr(image_path)

if not blurry:
    print("Image is clear and not blurry!!")
    embedding = DeepFace.represent(img_path = image_path, model_name = "Facenet")
    #live_emb = np.array(embedding)
    print("Face Embedding: ", embedding[0]["embedding"])                 #
else:
    print("Blurr image: will be discarded/ not saved")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\deriv.cpp:792: error: (-215:Assertion failed) !_src.empty() in function 'cv::Laplacian'


In [ ]:
live_emb = np.array(embedding[0]["embedding"])
Live_E = live_emb.reshape(1, -1)
print(Live_E)

In [ ]:
print(Live_E.shape)

In [ ]:
from clickhouse_driver import Client

client = Client('localhost')                                     # enable and connect database 
Result = "SELECT embedding FROM Data.Embeddings"                 # extact embeddings from the database and store them into results 
stored_data = client.execute(Result)                             

In [ ]:
final_Embed = np.array(stored_data)                              # convert the data into numpy array

In [ ]:
F_Emb = final_Embed.reshape(7, -1)                               # reshaping the data for seperate entry

In [ ]:
print(F_Emb)                                                     # print to check whether done correctly or not

In [ ]:
print(F_Emb.shape)

In [ ]:
threshold = 0.7
image_names = ["1.jpg", "2.jpg", "3.jpg", "4.jpg", "5.jpg", "6.jpg"]

#for embedding in F_Emb:
similarity = cosine_similarity(Live_E, F_Emb)                                      # check similarity for best match
similarities = similarity.flatten()                                                # use flatten to convert 2d array to 1d array
print(similarities)                                                                # print all the similarity score

In [ ]:
for i, similiarity in enumerate(similarities):
    if (similiarity>= threshold):
        print(f"Live capture matched with : {image_names[i]}, Similarity: {similiarity}")         # print the best matched image

In [ ]:

mark_attendance(image_names, similarity)                                          # mark the attandance.